# 트리 DP

트리 구조에서 DFS로 상태를 결합한다. 자식 노드 → 부모 노드로 상태를 누적하고, 상태를 정의한다.

- 서브트리 누적 : 한 정점을 루트로하는 서브트리 관리
- 선택/비선택 분기 : 선택 조건(인접 노드 간 제약 조건)이 있는 문제
- 루트 이동 : 모든 정점을 루트로하는 서브트리 관리

In [1]:
size = 7

tree = {
  0: [1, 2], 
  1: [0, 3, 4], 
  2: [0, 5, 6], 
  3: [1], 
  4: [1], 
  5: [2], 
  6: [2]
}

value = [10, 1, 5, 10, 5, 5, 5]

In [2]:
# 서브트리 누적 : DFS 하향식으로 자식 → 부모 상태 누적
def subtree_size(tree, node_size, root):
  size = [1 for _ in range(node_size)]
  parent = [-1 for _ in range(node_size)]

  order = []
  stack = [root]

  # DFS : post-order 탐색으로 부모 및 순서 기록
  while stack:
    node = stack.pop()
    order.append(node)
    for next in tree[node]:
      if next == parent[node]:
        continue
      parent[next] = node
      stack.append(next)

  # post-order의 역순 순회로 자식 → 부모 방향으로 서브트리 크기 누적
  for node in reversed(order):
    p = parent[node]
    if p != -1:
      size[p] += size[node]
  
  return size

size_list = subtree_size(tree, size, 0)
for i in range(size):
  print(f'서브트리 크기 (노드 {i} 시작): {size_list[i]}')

서브트리 크기 (노드 0 시작): 7
서브트리 크기 (노드 1 시작): 3
서브트리 크기 (노드 2 시작): 3
서브트리 크기 (노드 3 시작): 1
서브트리 크기 (노드 4 시작): 1
서브트리 크기 (노드 5 시작): 1
서브트리 크기 (노드 6 시작): 1


In [3]:
# 트리 독립 집합 : 노드를 기준으로 노드가 포함된 집합과 아닌 집합으로 나눔
def max_weight_tree_set(tree, value, size):
  parent = [-1 for _ in range(size)]
  include = [0 for _ in range(size)]
  exclude = [0 for _ in range(size)]
  
  order = []
  stack = [0]

  # DFS : post-order 탐색으로 부모 및 순서 기록
  while stack:
    node = stack.pop()
    order.append(node)
    for next in tree[node]:
      if next == parent[node]:
        continue    
      parent[next] = node
      stack.append(next)

  # post-order의 역순 순회로 자식 → 부모 방향으로 순회
  for node in reversed(order):
    include[node] = value[node]
    for next in tree[node]:
      if next == parent[node]:
        continue    
      exclude[node] += max(exclude[next], include[next]) # 본인 미선택 : 자식은 선택 or 미선택
      include[node] += exclude[next]                     # 본인 선택 : 자식은 미선택

  selected = []
  stack = [(0, False)]

  # 집합 내 값 탐색
  while stack:
    node, parent_selected = stack.pop()
    if parent_selected:
      for next in tree[node]:
        if next != parent[node]:
          stack.append((next, False))
    else:
      take = include[node] > exclude[node]
      if take:
        selected.append(node)
      for next in tree[node]:
        if next != parent[node]:
          stack.append((next, take))
  selected.sort()

  return max(include[0], exclude[0]), selected

weight, member = max_weight_tree_set(tree, value, size)
print(f"최대 독립 집합 값 : {weight} {member} ")

최대 독립 집합 값 : 35 [0, 3, 4, 5, 6] 


In [4]:
from collections import deque

# 루트 이동 : 2번의 DFS (1: 초기 상태 계산, 2: 상태 재귀 갱신)
def subtree_value_sum(tree, value, size, root=0):
  sub_weight = [0 for _ in range(size)]
  parent = [-1 for _ in range(size)]

  order = []
  stack = [root]

  # DFS : post-order 탐색으로 부모 및 순서 기록
  while stack:
    node = stack.pop()
    order.append(node)
    for next in tree[node]:
      if next == parent[node]:
        continue
      parent[next] = node
      stack.append(next)

  # post-order의 역순 순회로 자식 → 부모 방향으로 서브트리 크기 누적
  for node in reversed(order):
    sub_weight[node] = value[node]
    for next in tree[node]:
      if next == parent[node]:
        continue
      sub_weight[node] += sub_weight[next]

  total_sub_weight = sub_weight[root]
  weight = [0 for _ in range(size)]
  weight[root] = total_sub_weight

  # BFS : pre-order 탐색으로 부모 → 자식 방향으로 루트 이동 시 서브트리 크기 계산
  queue = deque([root])
  while queue:
    node = queue.popleft()
    for next in tree[node]:
      if next == parent[node]:
        continue
      # root == node에서 root == next로 변경된 상황
      # weight[node]에는 sub_weight[next]가 포함되어 있으므로 1번 제거
      # total_sub_weight에는 weight[next]가 포함되어 있으므로 1번 제거
      weight[next] = (weight[node] - sub_weight[next]) + (total_sub_weight - sub_weight[next])
      queue.append(next)

  return weight


weights = subtree_value_sum(tree, value, size)
for i in range(size):
  print(f"노드 {i}가 루트일 때 서브트리 합 : {weights[i]}")

노드 0가 루트일 때 서브트리 합 : 41
노드 1가 루트일 때 서브트리 합 : 50
노드 2가 루트일 때 서브트리 합 : 52
노드 3가 루트일 때 서브트리 합 : 71
노드 4가 루트일 때 서브트리 합 : 81
노드 5가 루트일 때 서브트리 합 : 83
노드 6가 루트일 때 서브트리 합 : 83
